<a href="https://colab.research.google.com/github/celaira/roof_deep_learning/blob/main/dida_testtask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mount google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import stuff

In [1]:
#import json
import os
import cv2 as cv
import numpy as np
#from PIL import Image
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
#from google.colab.patches import cv2_imshow

create path to data:

In [ ]:
training_image_path = '/content/drive/MyDrive/training_image'
label_path = '/content/drive/MyDrive/label'
test_image_path = '/content/drive/MyDrive/test_image'

import the images:

In [ ]:
def load_images_from_folder(folder):
    images = []
    filenames = sorted(os.listdir(folder))
    for filename in filenames:
        img = cv.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

train_image = np.array(load_images_from_folder(training_image_path))
label = np.array(load_images_from_folder(label_path))
test_image = np.array(load_images_from_folder(test_image_path))
#binarize the labels
label = np.where(label > 0.5, 1, 0)
label = np.array([cv.threshold(l, 127, 1, cv.THRESH_BINARY)[1] for l in label])